In [1]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, gwTxP):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --gatewayTxPriority="+f"{gwTxP}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os


def run_scenario(traffic_type, realistic_model, device_class, scenario_name, gwTxP):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, gwTxP)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_gw_tx_priority'
    test_sub_folder = ''

    if(gwTxP):
        test_sub_folder = 'True'
    else:
        test_sub_folder = 'False'

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []


In [7]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", False)))

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", False)))

threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", False)))

In [8]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", False)))

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", False)))

threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", False)))

In [9]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status GW_Tx_Priority', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_UNCONFIRMED_B, Simualtion with device count: 2
ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_MIXED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=Fal

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...


/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 2
ULPDR_UNCONFIRMED_B, Simualtion with device count: 4
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=B --appPeriod=600 --simu

+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
Computing performance metrics...


/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 2
ULPDR_UNCONFIRMED_B, Simualtion with device count: 2
ULPDR_MIXED_B, Simualtion with device count: 3
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=F

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
1.0 L: 0.2475
1.0 L: 0.30375
1.0 L: 0.3875
1.0 L: 0.45375
1.0 L: 0.5625
0.9946902654867257 L: 0.70625
0.9943019943019943 L: 0.8775
0.977725674091442 L: 1.06625
0.9887218045112782 L: 1.33
0.9703872437357631 L: 1.64625
0.9386763812993322 L: 2.05875
0.9001454192922927 L: 2.57875
0.845247446975648 L: 3.1825
0.7712665406427222 L: 3.9675
0.6783074177925057 L: 4.90375
0.5759270768593329 L: 6.03375
0.4714404662781016 L: 7.50625


Computing performance metrics...
Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
0.9 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
0.9705882352941176 L: 0.085
0.963855421686747 L: 0.10375
0.9791666666666666 L: 0.12
0.9834710743801653 L: 0.15125
0.9668874172185431 L: 0.18875
0.9895833333333334 L: 0.24
0.9208333333333333 L: 0.3
0.9664429530201343 L: 0.3725
0.9382022471910112 L: 0.445
0.9472477064220184 L: 0.545
0.9163636363636364 L: 0.6875
0.9214814814814815 L: 0.84375
0.9073405535499398 L: 1.03875
0.8946859903381642 L: 1.29375
0.865040650406504 L: 1.5375
0.8408370323398858 L: 1.97125
0.8264094955489614 L: 2.5275
0.8121069182389937 L: 3.18
0.775151708719259 L: 3.91375
0.7558953096657165 L: 4.82375
0.7018852289206546 L: 6.03375
0.6343047460449626 L: 7.50625


Computing performance metrics...
Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10125
1.0 L: 0.11875
1.0 L: 0.14875
0.9865771812080537 L: 0.18625
0.978021978021978 L: 0.2275
0.9781659388646288 L: 0.28625
0.9964285714285714 L: 0.35
0.9911504424778761 L: 0.42375
0.9744186046511628 L: 0.5375
0.96484375 L: 0.64
0.9707692307692307 L: 0.8125
0.9556962025316456 L: 0.9875
0.9246231155778895 L: 1.24375
0.9012048192771084 L: 1.55625
0.8263395739186572 L: 1.93625
0.7961717537506466 L: 2.41625
0.7808219178082192 L: 3.1025
0.7778138177100227 L: 3.85375
0.7357272022733143 L: 4.83875
0.6833920796184947 L: 6.02875
0.64729392173189 L: 7.50625
84710.39839172363
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
